# Классификатор тональности твитов

1. Скачиваем данные

In [1]:
#!pip install scikit-learn pandas

In [33]:
import pandas as pd

In [36]:
data_source_url = "https://github.com/satyajeetkrjha/kaggle-Twitter-US-Airline-Sentiment-/blob/master/Tweets.csv?raw=true"
corpus = pd.read_csv(data_source_url)

In [37]:
corpus.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


2. Преобразуем в векторное представление - будем использовать сырые частоты токенов

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X = count_vect.fit_transform(corpus.text)
X.shape

(14640, 15051)

Посмотрим, что получилось для конкретного текста:

In [39]:
X[0]  # sparse matrix - разреженная матрица (вспоминаем про data sparsity)

<1x15051 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [40]:
X[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

3. Заводим классификатор

In [41]:
from sklearn.naive_bayes import MultinomialNB

In [70]:
clf = MultinomialNB()
y = corpus.airline_sentiment
clf.fit(X, y)  # собственно обучение

MultinomialNB()

Посмотрим на параметры:

In [71]:
clf.coef_

array([[ -9.50266878,  -9.05068365, -12.14172611, ..., -11.44857893,
        -11.44857893, -11.44857893],
       [-10.28180412,  -9.36551339, -10.28180412, ..., -10.9749513 ,
        -10.9749513 , -10.9749513 ],
       [-10.75830705,  -8.81239691, -10.75830705, ..., -10.75830705,
        -10.75830705, -10.75830705]])

4. Оценим конкретный текст

In [68]:
import random
n = random.choice(range(5000))
print(corpus.text[n])

@VirginAmerica wish you flew out of Atlanta... Soon?


In [69]:
clf.predict(X[n])

array(['positive'], dtype='<U8')

5. Правильная оценка качества:
  - разделить на train/test
  - обучиться на train'е, оцениваться на test'е

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [75]:
X_train.shape

(9808, 15051)

In [76]:
X_test.shape

(4832, 15051)

In [77]:
clf.fit(X_train, y_train)

MultinomialNB()

In [79]:
y_pred = clf.predict(X_test)

In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7740066225165563